# SVM Predictions

author = c.magg <br>
date = October 26, 2020 <br>

## Add dependencies and imports

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd#
import cv2
import scipy
import logging as log
import skimage.segmentation as segmentation
from scipy.spatial.distance import cdist

In [ ]:
log.basicConfig(format='%(levelname)s:%(message)s', level=log.INFO)

In [ ]:
# add KidsBrainProject main folder to paths
sys.path.append(os.path.abspath('../../'))
sys.path.append(os.path.abspath('../utils/'))

In [ ]:
path_data = "../../Data/features_standardized/"

In [ ]:
from sklearn import svm
from sklearn.model_selection import KFold
import random
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
from SVMSearch import SVMSearch
from SVRPrediction import SVRPrediction
from SVMFeatures import SVMFeatures

In [ ]:
all_files = [ filename for filename in os.listdir(path_data) if filename.endswith( "csv" ) ]
all_files_error = [os.path.join(path_data,fn) for fn in all_files if 'error_metrics' in fn]
all_files_features = [os.path.join(path_data,fn) for fn in all_files if 'features' in fn]
len(all_files_error), len(all_files_features)

In [ ]:
[x.split('/features_')[-1].split('.')[0] for x in all_files_features]

In [ ]:
idx = 0
all_files_error[idx], all_files_features[idx]

In [ ]:
structures = ['Brain',
                           'CerebellPOSTYL',
                           'Cingulumleft',
                           'Cingulumright',
                           'Corpuscallosum',
                           'CTV',
                           'CTV1',
                           'CTV2',
                           'Fornix',
                           'GTV',
                           'Hypothalamus',
                           'PapezCircle',
                           'PTV1',
                           'PTV2',
                           'Scalp',
                           'TemporalLobeLt',
                           'TemporalLobeRt',
                           'ThalamusantL',
                           'ThalamusantR',
                           'Thalamusleft',
                           'Thalamusright']
C = [85, 99, 9, 1, 5, 13, 14, 13, 99, 6, 1, 99, 3, 21, 64, 74, 89, 1, 99, 1, 1]

In [ ]:
dict(zip(structures, C))

## SVR Prediction

In [ ]:
pred = SVRPrediction()

In [ ]:
np.shape(pred.X[0])

In [ ]:
mask = cv2.imread("../../Data/1/Segmentation/t0/Brain/1_t0_Brain_slice28.png", cv2.IMREAD_GRAYSCALE)
img = cv2.imread("../../Data/1/Segmentation/png/1_slice28.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(img)
plt.show()
plt.imshow(mask)
plt.show()

In [ ]:
features, tmp = SVMFeatures([img], preds=[mask]).calculate(standardize=True)

In [ ]:
np.shape(features.values[0])

In [ ]:
pred.X[0][0] == features.values[0]

In [ ]:
f = features.values[0].reshape(1,-1)
print(f.shape)
pred.regressions['Brain'].predict(f)

In [ ]:
pred.make_prediction(img, mask, 'Brain')

## SVM

best SVM was with jaccard distance

In [ ]:
search = SVMSearch(all_files_features, all_files_error)

In [ ]:
svrs = dict().fromkeys(structures)
for idx in range(len(search.X)):
    X = search.X[idx]
    y = search.y[idx][:,1]  # jaccard distance
    print(np.shape(X), np.shape(y))
    regr = svm.SVR(kernel='rbf', C=85, epsilon=0.1)
    regr.fit(X, y)
    svrs[structures[idx]]= regr

In [ ]:
sv = regr.support_vectors_
b = regr.intercept_
dual_coeff = regr.dual_coef_
gamma = regr._gamma
print("gamma = ", gamma)
print("b = ", b)
print("support vectors = ", sv)
print("dual_coeff = ", dual_coeff)

In [ ]:
def make_prediction(X, dual_coeff, sv, gamma, b):
    return np.sum(dual_coeff.T*np.exp(-gamma*(np.abs(sv-X)**2))) + b

In [ ]:
np.shape(X)

In [ ]:
test= X[0,:].reshape(1,-1)
test_gt = y[-1]
print(np.shape(test), np.shape(test_gt))

In [ ]:
make_prediction(test, dual_coeff, sv, gamma, b)

In [ ]:
regr.predict(test)[0]

In [ ]:
svrs["Brain"].predict(test)

In [ ]:
sys.getsizeof(svrs)